In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.impute import IterativeImputer as MICEImputer
from fancyimpute import SoftImpute #ha de instalarse en el prompt de anaconda con pip install fancyimpute
from sklearn.linear_model import BayesianRidge

# Ejemplo 1

In [3]:
df=pd.read_csv('BL-Flickr-Images-Book.csv')
df.head()

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


In [4]:
to_drop=['Edition Statement',
         'Corporate Author',
         'Corporate Contributors',
         'Former owner',
         'Engraver',
         'Contributors',
         'Issuance type',
         'Shelfmarks']
df.drop(columns=to_drop, inplace=True)
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


In [5]:
df['Identifier'].is_unique # chequea si el indice es unico

True

In [6]:
df = df.set_index('Identifier')
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
Identifier,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


In [7]:
df.loc[206] # accede a un registro usando su nuevo indice

Place of Publication                                               London
Date of Publication                                           1879 [1878]
Publisher                                                S. Tinsley & Co.
Title                                   Walter Forbes. [A novel.] By A. A
Author                                                              A. A.
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

In [8]:
df.iloc[0] # accede a un registro usando su posición

Place of Publication                                               London
Date of Publication                                           1879 [1878]
Publisher                                                S. Tinsley & Co.
Title                                   Walter Forbes. [A novel.] By A. A
Author                                                              A. A.
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

In [9]:
#muestra 10 registros tras una posición dada
df.loc[1905:, 'Date of Publication'].head(10)

Identifier
1905           1888
1929    1839, 38-54
2836           1897
2854           1865
2956        1860-63
2957           1873
3017           1866
3131           1899
4598           1814
4884           1820
Name: Date of Publication, dtype: object

In [10]:
# extrae 4 digitos cualesquiera al principio del string 'Date of Publication'
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
extr.head()

Identifier
206    1879
216    1868
218    1869
472    1851
480    1857
Name: Date of Publication, dtype: object

In [11]:
# convierte el objeto string a numérico (float en este caso)
df['Date of Publication'] = pd.to_numeric(extr)
df['Date of Publication'].dtype

dtype('float64')

In [12]:
# porcentaje de valores nulos
df['Date of Publication'].isnull().sum() / len(df)

0.11717147339205986

In [13]:
df['Date of Publication'].head(10)

Identifier
206     1879.0
216     1868.0
218     1869.0
472     1851.0
480     1857.0
481     1875.0
519     1872.0
667        NaN
874     1676.0
1143    1679.0
Name: Date of Publication, dtype: float64

In [14]:
df.loc[4157862]

Place of Publication                                  Newcastle-upon-Tyne
Date of Publication                                                1867.0
Publisher                                                      T. Fordyce
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4157862, dtype: object

In [15]:
df.loc[4159587]

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                1834.0
Publisher                                                Mackenzie & Dent
Title                   An historical, topographical and descriptive v...
Author                                              Mackenzie, E. (Eneas)
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4159587, dtype: object

In [16]:
pub = df['Place of Publication']
london = pub.str.contains('London')
london[:5]

Identifier
206    True
216    True
218    True
472    True
480    True
Name: Place of Publication, dtype: bool

In [17]:
# Limpia los valores de string de una columna en todas las filas
oxford = pub.str.contains('Oxford')
df['Place of Publication'] = np.where(london, 'London',
                                      np.where(oxford, 'Oxford',
                                               pub.str.replace('-', ' ')))
df['Place of Publication'].head()

Identifier
206    London
216    London
218    London
472    London
480    London
Name: Place of Publication, dtype: object

In [18]:
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
Identifier,,,,,,
206,London,1879.0,S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London,1868.0,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869.0,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851.0,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857.0,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


# Ejemplo 2

In [20]:
university_towns = []
with open ('university_towns.txt') as file:
    for line in file:
        if '[edit]' in line:
            # Remember this 'state' until the next is found 
            state = line
        else: 
            # Otherwise, we have a city; keep 'state' as last- seen
            university_towns.append ((state, line))
university_towns [:5]

[('Alabama[edit]\n', 'Auburn (Auburn University)[1]\n'),
 ('Alabama[edit]\n', 'Florence (University of North Alabama)\n'),
 ('Alabama[edit]\n', 'Jacksonville (Jacksonville State University)[2]\n'),
 ('Alabama[edit]\n', 'Livingston (University of West Alabama)[2]\n'),
 ('Alabama[edit]\n', 'Montevallo (University of Montevallo)[2]\n')]

In [21]:
towns_df = pd.DataFrame(university_towns, 
                         columns=['State', 'RegionName'])
towns_df.head()

,State,RegionName
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n


In [22]:
def get_citystate (item):
    if '(' in item:
        return item [:item.find('(')]
    elif '[' in item:
        return item [:item.find('[')]
    else:
        return item
towns_df= towns_df.map (get_citystate)
towns_df.head ()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


# Ejemplo 3

In [24]:
olympics_df = pd.read_csv ('olympics.csv')
olympics_df.head ()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,? Summer,01 !,02 !,03 !,Total,? Winter,01 !,02 !,03 !,Total,? Games,01 !,02 !,03 !,Combined total
1,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
2,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
3,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
4,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12


In [25]:
# omite la fila 0
olympics_df = pd.read_csv ('olympics.csv', header=1)
olympics_df.head()

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [26]:
new_names = {'Unnamed: 0' : 'Country', 
             '? Summer': 'Summer Olympics',
             '01 !' :'Gold',
             '02 !' :'Silver',
             '03 !' :'Bronze',
             '? Winter': 'Winter Olympics',
             '01 !.1' :'Gold.1',
             '02 !.1' :'Silver.1',
             '03 !.1' :'Bronze.1',
             '? Games': '#Games',
             '01 !.2' :'Gold.2',
             '02 !.2' :'Silver.2',
             '03 !.2' :'Bronze.2'} 
olympics_df.rename(columns=new_names, inplace=True)
olympics_df.head()

,Country,Summer Olympics,Gold,Silver,Bronze,Total,Winter Olympics,Gold.1,Silver.1,Bronze.1,Total.1,#Games,Gold.2,Silver.2,Bronze.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


# Ejercicio 1

In [28]:
#Crea un dataframe vacío
n_columns = 5
columns_names = ['C1','C2','C3','C4' ,'C5']
n_rows_complete = 5
n_rows_missing = 5
#Crea un array con numeros aleatorios entre 0 y 9 para las variables discretas
complete_data = np.random.randint(0,10, size = (n_rows_complete, n_columns))
#Crea un array lleno de NaN                                 
null_data = np.empty([n_rows_missing, n_columns])
null_data [:] = np.nan
#Concatena los dos arrays y los mete en el dataframe
full_data = np.concatenate((complete_data, null_data), axis=0)
df = pd.DataFrame(full_data, columns=columns_names)
df

,C1,C2,C3,C4,C5
0,7.0,6.0,4.0,9.0,0.0
1,4.0,9.0,5.0,2.0,8.0
2,6.0,6.0,7.0,3.0,4.0
3,9.0,0.0,0.0,1.0,7.0
4,8.0,0.0,6.0,0.0,4.0
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [29]:
#Reordena el dataframe aleatoriamente
full_data = np.random.permutation(full_data.flat)
full_data = full_data.reshape([n_rows_complete+ n_rows_missing, n_columns])
df = pd.DataFrame(full_data, columns=columns_names)
df

,C1,C2,C3,C4,C5
0,0.0,NaN,NaN,NaN,6.0
1,NaN,NaN,NaN,4.0,6.0
2,6.0,NaN,NaN,NaN,NaN
3,NaN,NaN,0.0,3.0,NaN
4,0.0,9.0,8.0,NaN,7.0
5,1.0,NaN,4.0,7.0,4.0
6,NaN,7.0,NaN,2.0,4.0
7,NaN,NaN,0.0,NaN,NaN
8,0.0,NaN,9.0,NaN,8.0
9,5.0,9.0,6.0,NaN,NaN


In [30]:
#Imputa utilizando imputador simple usando la estartegia de más frecuencia
imputer_mean = SimpleImputer(strategy='most_frequent')
imputed_data_simple = imputer_mean.fit_transform(full_data)
imputed_data_simple = pd.DataFrame(imputed_data_simple, columns= columns_names)
imputed_data_simple

,C1,C2,C3,C4,C5
0,0.0,9.0,0.0,2.0,6.0
1,0.0,9.0,0.0,4.0,6.0
2,6.0,9.0,0.0,2.0,4.0
3,0.0,9.0,0.0,3.0,4.0
4,0.0,9.0,8.0,2.0,7.0
5,1.0,9.0,4.0,7.0,4.0
6,0.0,7.0,0.0,2.0,4.0
7,0.0,9.0,0.0,2.0,4.0
8,0.0,9.0,9.0,2.0,8.0
9,5.0,9.0,6.0,2.0,4.0


In [31]:
#Imputa utilizando imputador K-nearest neighbours
imputer_KNN = KNNImputer()
imputed_data_KNN = imputer_KNN.fit_transform(full_data)
imputed_data_KNN= pd.DataFrame(imputed_data_KNN, columns=columns_names)
#aqui los redondea a enteros
imputed_data_KNN = np.round(imputed_data_KNN,0)
imputed_data_KNN

,C1,C2,C3,C4,C5
0,0.0,8.0,5.0,4.0,6.0
1,1.0,8.0,4.0,4.0,6.0
2,6.0,8.0,5.0,4.0,6.0
3,2.0,8.0,0.0,3.0,6.0
4,0.0,9.0,8.0,4.0,7.0
5,1.0,8.0,4.0,7.0,4.0
6,1.0,7.0,5.0,2.0,4.0
7,2.0,8.0,0.0,4.0,6.0
8,0.0,8.0,9.0,4.0,8.0
9,5.0,9.0,6.0,4.0,6.0


In [32]:
#Imputa utilizando RandomForestClassifier
imputer_RF = IterativeImputer(RandomForestClassifier())
imputed_data_RF = imputer_RF.fit_transform(full_data)
imputed_data_RF= pd.DataFrame(imputed_data_RF, columns=columns_names)
imputed_data_RF

,C1,C2,C3,C4,C5
0,0.0,9.0,8.0,4.0,6.0
1,0.0,9.0,8.0,4.0,6.0
2,6.0,7.0,0.0,3.0,4.0
3,6.0,7.0,0.0,3.0,4.0
4,0.0,9.0,8.0,4.0,7.0
5,1.0,9.0,4.0,7.0,4.0
6,6.0,7.0,0.0,2.0,4.0
7,6.0,7.0,0.0,3.0,4.0
8,0.0,9.0,9.0,4.0,8.0
9,5.0,9.0,6.0,7.0,4.0


In [33]:
#Imputa utilizando MICE
imputer_MICE = MICEImputer()
imputed_data_MICE = imputer_MICE.fit_transform(full_data)
imputed_data_MICE= pd.DataFrame(imputed_data_MICE, columns=columns_names)
#aqui los redondea a enteros
imputed_data_MICE = np.round(imputed_data_MICE,0)
imputed_data_MICE

,C1,C2,C3,C4,C5
0,0.0,8.0,5.0,4.0,6.0
1,2.0,8.0,4.0,4.0,6.0
2,6.0,9.0,4.0,4.0,4.0
3,2.0,6.0,0.0,3.0,3.0
4,0.0,9.0,8.0,4.0,7.0
5,1.0,8.0,4.0,7.0,4.0
6,2.0,7.0,4.0,2.0,4.0
7,2.0,6.0,0.0,4.0,3.0
8,0.0,10.0,9.0,4.0,8.0
9,5.0,9.0,6.0,4.0,5.0


# Ejercicio 2

In [35]:
#Crea un dataframe vacío
n_columns = 5
columns_names = ['C1','C2','C3','C4' ,'C5']
n_rows_complete = 5
n_rows_missing = 5
#Crea un array con numeros aleatorios entre 0 y 1 
complete_data = np.random.random([n_rows_complete, n_columns])
#Crea un array lleno de NaN                                 
null_data = np.empty([n_rows_missing, n_columns])
null_data [:] = np.nan
#Concatena los dos arrays y los mete en el dataframe
full_data = np.concatenate((complete_data, null_data), axis=0)
df = pd.DataFrame(full_data, columns=columns_names)
df

,C1,C2,C3,C4,C5
0,0.318414,0.275166,0.689031,0.802234,0.799444
1,0.009376,0.521651,0.926236,0.379665,0.005316
2,0.251645,0.408824,0.203446,0.733614,0.680546
3,0.772984,0.958962,0.332279,0.946668,0.977195
4,0.931859,0.683171,0.206903,0.828397,0.339814
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [36]:
#Reordena el dataframe aleatoriamente
full_data = np.random.permutation(full_data.flat)
full_data = full_data.reshape([n_rows_complete+ n_rows_missing, n_columns])
df = pd.DataFrame(full_data, columns=columns_names)
df

,C1,C2,C3,C4,C5
0,0.772984,0.379665,NaN,NaN,NaN
1,NaN,NaN,0.799444,0.689031,NaN
2,NaN,NaN,0.680546,NaN,NaN
3,0.683171,NaN,0.926236,NaN,NaN
4,NaN,0.339814,0.958962,0.332279,NaN
5,NaN,0.521651,0.251645,0.733614,0.203446
6,NaN,NaN,0.206903,0.946668,NaN
7,0.802234,NaN,NaN,0.275166,0.828397
8,NaN,NaN,0.977195,0.318414,0.005316
9,NaN,0.009376,0.408824,NaN,0.931859


In [37]:
#Imputa utilizando imputador simple usando la estartegia de media (otras psoibles estrategias pueden ser median, most_frequent o constant) 
imputer_mean = SimpleImputer(strategy='mean')
imputed_data_mean = imputer_mean.fit_transform(full_data)
imputed_data_mean = pd.DataFrame(imputed_data_mean, columns= columns_names)
imputed_data_mean

,C1,C2,C3,C4,C5
0,0.772984,0.379665,0.651219,0.549195,0.492254
1,0.752796,0.312627,0.799444,0.689031,0.492254
2,0.752796,0.312627,0.680546,0.549195,0.492254
3,0.683171,0.312627,0.926236,0.549195,0.492254
4,0.752796,0.339814,0.958962,0.332279,0.492254
5,0.752796,0.521651,0.251645,0.733614,0.203446
6,0.752796,0.312627,0.206903,0.946668,0.492254
7,0.802234,0.312627,0.651219,0.275166,0.828397
8,0.752796,0.312627,0.977195,0.318414,0.005316
9,0.752796,0.009376,0.408824,0.549195,0.931859


In [38]:
#Imputa utilizando imputador K-nearest neighbours
imputer_KNN = KNNImputer()
imputed_data_KNN = imputer_KNN.fit_transform(full_data)
imputed_data_KNN= pd.DataFrame(imputed_data_KNN, columns=columns_names)
imputed_data_KNN

,C1,C2,C3,C4,C5
0,0.772984,0.379665,0.550514,0.595351,0.492254
1,0.752796,0.312627,0.799444,0.689031,0.492254
2,0.752796,0.312627,0.680546,0.604001,0.492254
3,0.683171,0.312627,0.926236,0.469701,0.492254
4,0.752796,0.339814,0.958962,0.332279,0.492254
5,0.752796,0.521651,0.251645,0.733614,0.203446
6,0.752796,0.312627,0.206903,0.946668,0.492254
7,0.802234,0.312627,0.669022,0.275166,0.828397
8,0.752796,0.312627,0.977195,0.318414,0.005316
9,0.752796,0.009376,0.408824,0.595351,0.931859


In [39]:
#Imputa utilizando ExtraTreesRegressor
imputer_ET = IterativeImputer(ExtraTreesRegressor())
imputed_data_ET = imputer_ET.fit_transform(full_data)
imputed_data_ET= pd.DataFrame(imputed_data_ET, columns=columns_names)
imputed_data_ET

,C1,C2,C3,C4,C5
0,0.772984,0.379665,0.417399,0.545747,0.237826
1,0.706522,0.438877,0.799444,0.689031,0.092493
2,0.731670,0.450758,0.680546,0.690436,0.134800
3,0.683171,0.341010,0.926236,0.342981,0.007297
4,0.683171,0.339814,0.958962,0.332279,0.005316
5,0.781132,0.521651,0.251645,0.733614,0.203446
6,0.781132,0.521651,0.206903,0.946668,0.203446
7,0.802234,0.034101,0.407794,0.275166,0.828397
8,0.683171,0.333205,0.977195,0.318414,0.005316
9,0.802234,0.009376,0.408824,0.275166,0.931859


In [40]:
#Imputa utilizando MICE
imputer_MICE = MICEImputer()
imputed_data_MICE = imputer_MICE.fit_transform(full_data)
imputed_data_MICE= pd.DataFrame(imputed_data_MICE, columns=columns_names)
imputed_data_MICE

,C1,C2,C3,C4,C5
0,0.772984,0.379665,0.671319,0.544017,0.464504
1,0.675689,0.556997,0.799444,0.689031,0.427799
2,0.773522,0.369350,0.680546,0.553196,0.489197
3,0.683171,0.323347,0.926236,0.424736,0.583215
4,0.657744,0.339814,0.958962,0.332279,0.510767
5,0.903817,0.521651,0.251645,0.733614,0.203446
6,0.966467,0.435877,0.206903,0.946668,0.445946
7,0.802234,0.002646,0.863783,0.275166,0.828397
8,0.512798,0.751616,0.977195,0.318414,0.005316
9,1.014296,0.009376,0.408824,0.751627,0.931859


In [41]:
#Imputa utilizando SoftImpute
imputer_SI = SoftImpute(verbose=False)
imputed_data_SI = imputer_SI.fit_transform(full_data)
imputed_data_SI= pd.DataFrame(imputed_data_SI, columns=columns_names)
imputed_data_SI

,C1,C2,C3,C4,C5
0,0.772984,0.379665,0.382581,0.199098,0.439022
1,0.231880,0.378865,0.799444,0.689031,0.161498
2,0.190301,0.151919,0.680546,0.177330,0.072092
3,0.683171,0.338593,0.926236,0.249069,0.392355
4,0.371786,0.339814,0.958962,0.332279,0.106457
5,0.346827,0.521651,0.251645,0.733614,0.203446
6,0.068423,0.416242,0.206903,0.946668,0.161174
7,0.802234,0.250798,0.367678,0.275166,0.828397
8,0.218597,0.257433,0.977195,0.318414,0.005316
9,0.565757,0.009376,0.408824,0.192104,0.931859


In [42]:
#Imputa utilizando BayesianRidge
imputer_BR = IterativeImputer(BayesianRidge())
imputed_data_BR = imputer_BR.fit_transform(full_data)
imputed_data_BR= pd.DataFrame(imputed_data_BR, columns=columns_names)
imputed_data_BR

,C1,C2,C3,C4,C5
0,0.772984,0.379665,0.671319,0.544017,0.464504
1,0.675689,0.556997,0.799444,0.689031,0.427799
2,0.773522,0.369350,0.680546,0.553196,0.489197
3,0.683171,0.323347,0.926236,0.424736,0.583215
4,0.657744,0.339814,0.958962,0.332279,0.510767
5,0.903817,0.521651,0.251645,0.733614,0.203446
6,0.966467,0.435877,0.206903,0.946668,0.445946
7,0.802234,0.002646,0.863783,0.275166,0.828397
8,0.512798,0.751616,0.977195,0.318414,0.005316
9,1.014296,0.009376,0.408824,0.751627,0.931859
